In [1]:
import gc
from src.features_generation import feature_engineering

X, y, X_test, sample_submission = feature_engineering()

gc.collect()

Timer unit: 1e-06 s

Total time: 86.0245 s
File: /workdir/src/features_generation.py
Function: feature_engineering at line 221

Line #      Hits         Time  Per Hit   % Time  Line Contents
   221                                           @profile
   222                                           def feature_engineering():
   223                                               """
   224                                               load original datasets and conduct feature engineering
   225                                               :return X_train, y_train, X_test, submission:
   226                                               """
   227         1   21810940.0 21810940.0     25.4      train_identity, train_transaction, test_identity, test_transaction, submission = load_data(DATA_DIRECTORY)
   228                                           
   229         1    6747236.0 6747236.0      7.8      train_identity = id_split(train_identity)
   230         1    6811199.0 6811199.0      7

0

In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import numpy as np
import pandas as pd

In [3]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
         }


In [4]:
%%time

NFOLDS = 5
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X.shape[0])
score = 0

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500)
    
    feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
    
    score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
    y_preds += clf.predict(X_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()
    
print(f"\nMean AUC = {score}")
print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")

Training until validation scores don't improve for 500 rounds
[200]	training's auc: 0.948771	valid_1's auc: 0.8848
[400]	training's auc: 0.969448	valid_1's auc: 0.898816
[600]	training's auc: 0.982939	valid_1's auc: 0.907714
[800]	training's auc: 0.989967	valid_1's auc: 0.913103
[1000]	training's auc: 0.993577	valid_1's auc: 0.915619
[1200]	training's auc: 0.995713	valid_1's auc: 0.916977
[1400]	training's auc: 0.996984	valid_1's auc: 0.917597
[1600]	training's auc: 0.997873	valid_1's auc: 0.917664
[1800]	training's auc: 0.998451	valid_1's auc: 0.91765
[2000]	training's auc: 0.998875	valid_1's auc: 0.917618
[2200]	training's auc: 0.999182	valid_1's auc: 0.917413
Early stopping, best iteration is:
[1763]	training's auc: 0.998357	valid_1's auc: 0.917801
Fold 1 | AUC: 0.9178013991058426
Training until validation scores don't improve for 500 rounds
[200]	training's auc: 0.948772	valid_1's auc: 0.906949
[400]	training's auc: 0.969041	valid_1's auc: 0.917841
[600]	training's auc: 0.983299	va